#### iris 품종 분류 모델
    - 데이터 셋 : irirs.csv

In [4]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torchmetrics.classification import F1Score
import torch.nn.functional as F
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader


In [5]:
# 기기 설정 (GPU or CPU)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
# 사용자 정의 Dataset 클래스 : feature와 target을 텐서로 변환
class MyDataset(Dataset):
    def __init__(self,features,targets):
        self.features = torch.FloatTensor(features) # 피쳐 텐서화
        self.targets = torch.FloatTensor(targets) # 타겟 텐서화

    def __len__(self):
        return len(self.targets)
    def __getitem__(self,idx):
        return self.features[idx], self.targets[idx]
    
# 모델 클래스

class MyModel(nn.Module):
    def __init__(self,in_features, out_target,hidden_layers,output_activation=None):

        '''
        in_features : 입력 피쳐 수
        out_target : 출력 노드 수( 다중분류 = 클래스 수)
        hidden_layer : 은닉층 노드 수를 리스트로 받아서 은닉층의 개수와 각 층의 노드 수를 설정
        output_activation : 출력층에 사용할 활성화 함수(None,'si)
        '''
        super(MyModel, self).__init__()

        self.input_layer = nn.Linear(in_features,hidden_layers[0])
        self.hidden_layer = nn.ModuleList()
        for i in range(len(hidden_layers)-1):
            self.hidden_layer.append(nn.Linear(hidden_layers[i],hidden_layers[i+1]))

        self.output_layer = nn.Linear(hidden_layers[-1],out_target)

        # 출력층 활성화 함수 설정 (None, 'sigmoid','softmax')
        self.output_activation = output_activation

    def forward(self,X):
        y = self.input_layer(X)
        y = F.relu(y)

        for layer in self.hidden_layer:
            y = layer(y)
            y = F.relu(y)
        
        y = self.output_layer(y)

        if self.output_activation == 'sigmoid':
            y = torch.sigmoid(y)
        elif self.output_activation == 'softmax':
            y = F.softmax(y,dim=1)

        return y

In [ ]:
# 학습용 함수
def epochTraining(model,train_loader,lossfunc,optimizer):
    model.train()
    loss_total = 0
    score_total = 0
    f1_metric = F1Score().to(DEVICE)
    for X_train, y_train in train_loader:
        X_train, y_train = X_train.to(DEVICE), y_train.to(DEVICE)
        pre_y = model(X_train)
        loss = lossfunc(pre_y,y_train)
        loss_total += loss.item()
        score_total += f1_metric(pre_y,y_train).item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return loss_total / len(train_loader), score_total / len(train_loader)

# 검증용 함수
def testing(model,val_loader,lossfunc):
    model.eval()
    loss_total = 0
    score_total = 0
    r2_metric = F1Score().to(DEVICE) # F1Score 분류형
    with torch.no_grad():
        for X_val, y_val in val_loader:
            X_val, y_val = X_val.to(DEVICE), y_val.to(DEVICE)
            pre_y = model(X_val)
            loss = lossfunc(pre_y,y_val)
            loss_total += loss.item()
            
